In [77]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

#  Download "glove-twitter-25" embeddings
glove_vectors = gensim.downloader.load('glove-twitter-25')
glove_vectors.most_similar('twitter')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[==================================================] 100.0% 104.8/104.8MB downloaded


[('facebook', 0.948005199432373),
 ('tweet', 0.9403423070907593),
 ('fb', 0.9342359900474548),
 ('instagram', 0.9104824066162109),
 ('chat', 0.8964963555335999),
 ('hashtag', 0.8885936737060547),
 ('tweets', 0.8878158330917358),
 ('tl', 0.8778461217880249),
 ('link', 0.877821147441864),
 ('internet', 0.8753897547721863)]

In [70]:
import numpy as np
import tensorflow as tf
import time
import pandas as pd
import json

print(time.strftime("%H:%M:%S", time.gmtime()))

# Import amazon reviews dataset 

filepath = '/Users/p.mittal/Library/Mobile Documents/com~apple~CloudDocs/Roux/Courses/NLP/Assignments/Assignment 3 SA/Amazon Reviews/reviews_Office_Products_5.json'

data = []
with open(filepath
            , 'r') as f:
        for line in f:
            data.append(json.loads(line))

df = pd.DataFrame(data)


df["length_review"] = df["reviewText"].apply(lambda x: len(x.split()))
df["length_summary"] = df["summary"].apply(lambda x: len(x.split()))

print(df["reviewText"][0])
df.head()


# SUbset 1000 reviews for faster processing
df2  = df.sample(1000)





21:18:24
I bought my first HP12C in about 1984 or so, and it served me faithfully until 2002 when I lost it while travelling.  I searched for another one to replace it, but found one difficult to come by in my area.  So, I decided to buy up and purchased an HP 49G.  What a mistake!  I know that many people view the HP 49G (now 49G+) as the flagship of the HP line, but for me that was a disaster.The 49G may be powerful, but using it is extremely counterintuitive...and the manual was sketchy at best.  The 12C, on the other hand, does what I need in a way that makes good sense to me.If you are looking for a solid, reliable calculator, the HP12C may be for you.  It's programmable.  It does basic statistics well, and many business applications too.  The manual makes sense; you will be up and running in short order.I'm ready to set my 49G aside and move back to my old friend.  I didn't even have to replace the batteries in well over a decade of use!HP 12C, I'm coming home!


In [71]:
# Preprocessing the data
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    text = word_tokenize(text)
    text = [word for word in text if word not in stopwords.words('english')]
    # text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

df2["reviewText2"] = df2["reviewText"].apply(lambda x: clean_text(x))
df2["summary2"] = df2["summary"].apply(lambda x: clean_text(x))

df2.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,length_review,length_summary,reviewText2,summary2
40122,A3KZEGBTPH6MMF,B006IB35GG,"Lucy Cat ""Mandy""","[0, 0]",I am a bit of a pen connoisseur -- I've litera...,3.0,Cheap feeling barrel - underwhelming performance,1331683200,"03 14, 2012",159,6,bit pen connoisseur ive literally tried spent ...,cheap feeling barrel underwhelming performance
42089,A3EP26M3QHINXT,B00886N6YA,Nathan G,"[0, 0]",I use this not for the RFID but for the protec...,5.0,Card cover,1357171200,"01 3, 2013",37,2,use rfid protection notice however blocked sca...,card cover
7610,A16PYNWJEUN5TL,B0008GQ26C,"SANDYEGGO ""SALLY GOSSELIN""","[0, 0]",Love the refills in Medium. I have always bou...,5.0,LOVE THE MEDIUM REFILLS!!,1385769600,"11 30, 2013",77,4,love refills medium always bought fine found s...,love medium refills
20232,A3EZEP0FX5BC1P,B00290N0RO,"A Conrad ""Ask Conrad""","[0, 0]",This is a great sketchbook. The papers are th...,5.0,Great SketchBook!,1403827200,"06 27, 2014",113,2,great sketchbook papers thick textured daughte...,great sketchbook
13503,A2065HBMYDXJ1S,B0010T3QT2,"Jenn B ""Happy Mom""","[0, 0]","These are good enevelopes, convenient to use. ...",4.0,Good product,1278979200,"07 13, 2010",33,2,good enevelopes convenient use sometimes pull ...,good product


In [72]:
# print the first review and its cleaned version
print(df2["reviewText"].reset_index(drop=True)[0])
print(df2["reviewText2"].reset_index(drop=True) [0])



I am a bit of a pen connoisseur -- I've literally tried them all (and spent way too much money in the process).  For ballpoint pens, its important that the ink runs smooth and consistently (I hate when the ink density changes mid-stroke.)  For all pens, the barrel needs to be comfortable and sturdy in your grip.  So, how do these "InkJoy"'s stack up?  For me they are just "O.K."  The ink runs smooth and is very consistent for a ball point, but the construction of the pen itself feels extremely cheap.  The barrel is literally bends when squeezed between my index finger and thumb and the pen tip itself feels/looks cheap.  I find these to be a little thick in diameter for my grip as well.  Overall, not a terribly impressive pen.  I have relegated these to the pens I keep out in the open at my office.  If someone walks off with them I wouldn't be upset....
bit pen connoisseur ive literally tried spent way much money process ballpoint pens important ink runs smooth consistently hate ink den

In [73]:
# Create word embeddings using word2vec for the cleaned reviews
from gensim.models import Word2Vec

# Create a list of list of words
sentences = [review.split() for review in df2["reviewText2"]]
print(sentences[0])

# Create the word2vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# # Save the model
# model.save("word2vec.model")

# # Load the model
# model = Word2Vec.load("word2vec.model")

# Get the word vector for a word
print(model.wv['good'])

# Get the most similar words
print(model.wv.most_similar('good'))

# print total number of words in the model
print(len(model.wv.index_to_key))

['bit', 'pen', 'connoisseur', 'ive', 'literally', 'tried', 'spent', 'way', 'much', 'money', 'process', 'ballpoint', 'pens', 'important', 'ink', 'runs', 'smooth', 'consistently', 'hate', 'ink', 'density', 'changes', 'midstroke', 'pens', 'barrel', 'needs', 'comfortable', 'sturdy', 'grip', 'inkjoys', 'stack', 'ok', 'ink', 'runs', 'smooth', 'consistent', 'ball', 'point', 'construction', 'pen', 'feels', 'extremely', 'cheap', 'barrel', 'literally', 'bends', 'squeezed', 'index', 'finger', 'thumb', 'pen', 'tip', 'feelslooks', 'cheap', 'find', 'little', 'thick', 'diameter', 'grip', 'well', 'overall', 'terribly', 'impressive', 'pen', 'relegated', 'pens', 'keep', 'open', 'office', 'someone', 'walks', 'wouldnt', 'upset']
[-0.483687    0.6683833   0.43632442 -0.08098825 -0.01126154 -1.2808864
  0.34584472  1.6455834  -0.38377935 -0.50351876 -0.31598938 -1.1642517
 -0.01870729  0.12697385  0.20496343 -0.46728417  0.13358793 -0.88426554
 -0.04302572 -1.4824363   0.1142078   0.28431258  0.33795682 -0.

In [74]:
# Then the word2vec embeddings are averaged for each review
# The average word2vec embeddings are used as input to a classifier 

# # Create the average word2vec embeddings for each review
# def get_average_word2vec_embeddings(review, model):
#     words = review.split()
#     embeddings = [model.wv[word] for word in words if word in model.wv.key_to_index]
#     if len(embeddings) == 0:
#         return np.zeros(model.vector_size)
#     return np.mean(embeddings, axis=0)

# df2["average_word2vec_embeddings"] = df2["reviewText2"].apply(lambda x: get_average_word2vec_embeddings(x, model))
# df2.head()


# Create the average word2vec embeddings for each review and save them to the dataframe as columns  
def get_average_word2vec_embeddings(review, model):
    words = review.split()
    embeddings = [model.wv[word] for word in words if word in model.wv.key_to_index]
    if len(embeddings) == 0:
        return [0]*model.vector_size
    return list(np.mean(embeddings, axis=0))


# save the embeddings as columns in the dataframe
for i in range(model.vector_size):
    df2["w2v_emb_" + str(i)] = df2["reviewText2"].apply(lambda x: get_average_word2vec_embeddings(x, model)[i])

df2.head()



<ipython-input-74-0bc46b168efa>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2["w2v_emb_" + str(i)] = df2["reviewText2"].apply(lambda x: get_average_word2vec_embeddings(x, model)[i])
<ipython-input-74-0bc46b168efa>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2["w2v_emb_" + str(i)] = df2["reviewText2"].apply(lambda x: get_average_word2vec_embeddings(x, model)[i])
<ipython-input-74-0bc46b168efa>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,length_review,...,w2v_emb_90,w2v_emb_91,w2v_emb_92,w2v_emb_93,w2v_emb_94,w2v_emb_95,w2v_emb_96,w2v_emb_97,w2v_emb_98,w2v_emb_99
40122,A3KZEGBTPH6MMF,B006IB35GG,"Lucy Cat ""Mandy""","[0, 0]",I am a bit of a pen connoisseur -- I've litera...,3.0,Cheap feeling barrel - underwhelming performance,1331683200,"03 14, 2012",159,...,0.314545,0.174229,0.029583,0.094286,0.506055,0.265732,0.159431,-0.336931,0.109561,-0.018319
42089,A3EP26M3QHINXT,B00886N6YA,Nathan G,"[0, 0]",I use this not for the RFID but for the protec...,5.0,Card cover,1357171200,"01 3, 2013",37,...,0.291546,0.162226,0.025777,0.084267,0.468024,0.248007,0.146568,-0.312164,0.103209,-0.019049
7610,A16PYNWJEUN5TL,B0008GQ26C,"SANDYEGGO ""SALLY GOSSELIN""","[0, 0]",Love the refills in Medium. I have always bou...,5.0,LOVE THE MEDIUM REFILLS!!,1385769600,"11 30, 2013",77,...,0.336221,0.190737,0.032658,0.103419,0.545792,0.284632,0.169230,-0.362457,0.115924,-0.018728
20232,A3EZEP0FX5BC1P,B00290N0RO,"A Conrad ""Ask Conrad""","[0, 0]",This is a great sketchbook. The papers are th...,5.0,Great SketchBook!,1403827200,"06 27, 2014",113,...,0.355736,0.200301,0.035174,0.108587,0.573154,0.304417,0.180462,-0.381223,0.121036,-0.022490
13503,A2065HBMYDXJ1S,B0010T3QT2,"Jenn B ""Happy Mom""","[0, 0]","These are good enevelopes, convenient to use. ...",4.0,Good product,1278979200,"07 13, 2010",33,...,0.470854,0.263674,0.046560,0.140443,0.757075,0.397847,0.240332,-0.502891,0.161757,-0.028047


In [85]:
# Sentiment Analysis using the word2vec embeddings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# convert Overall column into binary sentiment  1 for positive and 0 for negative 
# 1 and 2 are negative and 4 and 5 are positive, category 3 is removed from the data 
# start by removing the rows with overall 3
df2 = df2[df2["overall"] != 3]

# convert the overall column into binary sentiment
df2["response"] = df2["overall"].apply(lambda x: 1 if x > 3 else 0)

df2["response"].value_counts()

# print list of columns with the column number
# for i, col in enumerate(df2.columns):
#     print(i, col)


# Split the data into training and testing sets. Features are the word2vec embeddings and the target is the response column
X = df2.iloc[:, 13:113]
y = df2["response"]
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(time.strftime("%H:%M:%S", time.gmtime()))



Accuracy: 0.9351351351351351
12:15:21


/opt/anaconda3/envs/keras2/lib/python3.9/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/anaconda3/envs/keras2/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/keras2/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/keras2/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check 